# Async Query Demo

In [1]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes.
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.
import nest_asyncio

nest_asyncio.apply()

In [2]:
import time
from llama_index import ListIndex, SimpleDirectoryReader

In [3]:
query_str = "What is Paul Graham's biggest achievement?"

In [4]:
# load documents
documents = SimpleDirectoryReader("../paul_graham_essay/data").load_data()

In [5]:
index = ListIndex.from_documents(documents)

#### By default, generate a response through hierarchical tree summarization (i.e., `response_mode=tree_summarize`) makes blocking LLM calls

In [10]:
start_time = time.perf_counter()
query_engine = index.as_query_engine(response_mode="tree_summarize")
query_engine.query(query_str)
elapsed_time = time.perf_counter() - start_time

print(f"{elapsed_time:0.3f}s")

39.737s


It takes a long time to generate a response through hierarchical tree summarization (i.e., `response_mode=tree_summarize`). This is because each LLM call is waiting for the previous one to finish. Time is waisted in waiting for an IO response.  

With `async` call, instead of waiting for the response from the server, you carry on with the next requests, effectively batching together all the requests so that they can be done in parallel.

#### Option 1: Running `aquery` (async query call) will take advantage of async LLM calls

In [11]:
import asyncio

start_time = time.perf_counter()
task = query_engine.aquery(query_str)
asyncio.run(task)
elapsed_time = time.perf_counter() - start_time

print(f"{elapsed_time:0.3f}s")

13.021s


It is faster to generate a response through `aquery` (i.e., `response_mode=tree_summarize`).

#### Option 2: Pass in `use_async=True` to enable asynchronous LLM calls within a synchronous `query`

This approach makes a synchronous `query` calls, but runs async tasks during the "tree_summarize" operation.

In [12]:
start_time = time.perf_counter()
query_engine = index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
query_engine.query(query_str)
elapsed_time = time.perf_counter() - start_time

print(f"{elapsed_time:0.3f}s")

10.589s


It takes ~6.9s to generate a response through hierarchical tree summarization (i.e., `response_mode=tree_summarize`).

## Async Query with a list of Queries
now suppose you wanted to query the `query_engine` with a list of queries. While earlier the bottleneck was llama_index internally waiting for each response, here the bottleneck is when you wait for the response for each call `query()`.

In [16]:
# a list of different queries (yeah I cheated in this part)
query_list = [query_str] * 3

start_time = time.perf_counter()
query_engine = index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
for q in query_list:
    _ = query_engine.query(q)
elapsed_time = time.perf_counter() - start_time

print(f"{elapsed_time:0.3f}s")

35.264s


Here async can help you.

In [25]:
start_time = time.perf_counter()
query_engine = index.as_query_engine(
    response_mode="tree_summarize",
)

# run each query in parallel
async def async_query(query_engine, questions):
    tasks = [query_engine.aquery(q) for q in questions]
    r = await asyncio.gather(*tasks)
    return r


_ = asyncio.run(async_query(query_engine, query_list))
elapsed_time = time.perf_counter() - start_time

print(f"{elapsed_time:0.3f}s")

13.025s
